In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt


import seaborn as sns
from datetime import timedelta
import os
import write_data_table


pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [5]:
att_main_tw = pd.read_excel('Attribute_and_data_points.xlsx', sheet_name='Attributes_Taiwan')
att_main_tw.rename(columns={'SKU Description':'sku_description','Level2':'ph_level2','sku type':'item_type','units':'uom'},inplace=True)
att_main_tw              

,sku_description,sku,item_type,size,uom,cases,ph_level2,shortname
0,FAB ULTRAR CRM 56.7G(2.0OZ)X6X13 TW,80692947,Cream,2.0,Oz,78,Ultra Repair,URC*3
1,FAB CLEANSER EUP 142G(50Z)X57 NONSTK,80702886,Cleanser,5.0,Oz,57,NaN,Cleanser
2,FAB ULT OILC MOIST 50ML(1.7OZ)X6X8,80710476,Moistrizer,1.7,Oz,48,Ultra,OCC
3,FAB ULTRAT SERUM P 50ML(1.7OZ)X3X23,82332207,Serum,1.7,Oz,69,Ultra Repair,Serum
4,FAB ULTRAR MASK 56.7G(2.0OZ)X42 NONSTK,82332208,Cream,2.0,Oz,42,Ultra Repair,Oatmeal Mask
5,FAB ULTRAR TONER P 177ML(60Z)X6X8 NONSTK,82332219,Toner,6.0,Oz,48,Ultra Repair,Toner
6,FAB ULTRAR TONER P 177ML(6OZ)X6X8 NONSTK,82332800,Toner,6.0,Oz,48,Ultra Repair,Radiance Pad
7,FAB ULTRAR CRM 56.7G(2.0OZ)X6X13 TW,82332932,Cream,2.0,Oz,78,Ultra Repair,URC


In [17]:
att_main_china = pd.read_excel('Attribute_and_data_points.xlsx', sheet_name='Attributes_China')
att_main_china.rename(columns={'SKU Description':'sku_description','Level2':'ph_level2','sku type':'item_type','units':'uom'},inplace=True)
att_main_china.head()             

,sku_description,sku,item_type,size,uom,cases,ph_level2,shortname,material
0,FAB UT RPAR CRM 2.0OZ 56.7GX24 JAR JUL20,815517026036,Cream,56.7,G,24,Ultra Repair,Facial Radiance Pads 28 pcs,82314745
1,FAB UT RPAR CRM 4.0OZ 113GX24 JAR JUL20,815517026265,Cream,113.0,G,24,Ultra Repair,URC 4oz Jar,82314746
2,FAB ULTRA REPAIR CRM 6.0OZ 170GX12 JUL20,815517026005,Cream,170.0,G,12,Ultra Repair,URC 6oz Jar,82314748
3,FAB UT RP HYDR SERUM 1.7OZ 50MLX24 SEP20,815517026050,serum,50.0,ML,24,Ultra Repair,Hydrating Serum 1.7oz,82314724
4,FAB UTRP OAT TNR 6.0OZ177MLX12 JUL20,815517026067,mask,177.0,ML,12,Ultra Repair,Oatmeal Toner 6oz,82314750


In [56]:
buffer_plan_china = pd.read_excel('Replenishment_China.xlsx', sheet_name='Buffer Plan')
buffer_plan_china =  buffer_plan_china[[ 'site_name', 'item_name', 'period_start_date', 'Supply', 'Demand',   'Eoh','Type']]
buffer_plan_china.rename(columns = {'item_name' : 'sku'}, inplace = True)
buffer_plan_china = buffer_plan_china.melt(id_vars=[ 'sku','site_name', 'period_start_date', 'Type'],var_name="metric_name",value_name="metric_value")
buffer_plan_china['date'] = buffer_plan_china['period_start_date']
buffer_plan_china = pd.merge(buffer_plan_china, att_main_china, on = 'sku')

buffer_plan_china['site_name_main'] = buffer_plan_china['site_name'] 
buffer_plan_china.loc[buffer_plan_china['site_name'].str.contains('PG'),'site_name_main'] = 'P&G'
buffer_plan_china.loc[buffer_plan_china['site_name'].str.contains('DY'),'site_name_main'] = 'DY'
buffer_plan_china.loc[buffer_plan_china['site_name'].str.contains('JD'),'site_name_main'] = 'JD'
buffer_plan_china.loc[buffer_plan_china['site_name'].str.contains('TMALL'),'site_name_main'] = 'TMALL'
buffer_plan_china.head()

,sku,site_name,period_start_date,Type,metric_name,metric_value,date,sku_description,item_type,size,uom,cases,ph_level2,shortname,material,site_name_main
0,815517025992,Chengdu_JD,2022-07-01,Off-take,Supply,161.0,2022-07-01,FAB FACE CLEANSER 5.0OZ 142GX24 NOV21,cleanser,142.0,G,24,Face,Not found please neglect,80693598,JD
1,815517025992,Chengdu_JD,2022-08-01,Off-take,Supply,0.0,2022-08-01,FAB FACE CLEANSER 5.0OZ 142GX24 NOV21,cleanser,142.0,G,24,Face,Not found please neglect,80693598,JD
2,815517025992,Chengdu_JD,2022-09-01,Off-take,Supply,0.0,2022-09-01,FAB FACE CLEANSER 5.0OZ 142GX24 NOV21,cleanser,142.0,G,24,Face,Not found please neglect,80693598,JD
3,815517025992,Chengdu_JD,2022-10-01,Off-take,Supply,0.0,2022-10-01,FAB FACE CLEANSER 5.0OZ 142GX24 NOV21,cleanser,142.0,G,24,Face,Not found please neglect,80693598,JD
4,815517025992,Chengdu_JD,2022-11-01,Off-take,Supply,0.0,2022-11-01,FAB FACE CLEANSER 5.0OZ 142GX24 NOV21,cleanser,142.0,G,24,Face,Not found please neglect,80693598,JD


In [57]:
buffer_plan_taiwan = pd.read_excel('Replenishment Taiwan.xlsx', sheet_name='buffer plan')
buffer_plan_taiwan =  buffer_plan_taiwan[[ 'site_name', 'item_name', 'period_start_date', 'Supply', 'Demand',   'Eoh','Type']]
buffer_plan_taiwan.rename(columns = {'item_name' : 'sku'}, inplace = True)
buffer_plan_taiwan = buffer_plan_taiwan.melt(id_vars=[ 'sku','site_name', 'period_start_date', 'Type'],var_name="metric_name",value_name="metric_value")
buffer_plan_taiwan['date'] = buffer_plan_taiwan['period_start_date']
buffer_plan_taiwan = pd.merge(buffer_plan_taiwan, att_main_tw, on = 'sku')
buffer_plan_taiwan.head()

,sku,site_name,period_start_date,Type,metric_name,metric_value,date,sku_description,item_type,size,uom,cases,ph_level2,shortname
0,80692947,MOMO,2022-07-01,Off-take,Supply,466.0,2022-07-01,FAB ULTRAR CRM 56.7G(2.0OZ)X6X13 TW,Cream,2.0,Oz,78,Ultra Repair,URC*3
1,80692947,MOMO,2022-08-01,Off-take,Supply,14.0,2022-08-01,FAB ULTRAR CRM 56.7G(2.0OZ)X6X13 TW,Cream,2.0,Oz,78,Ultra Repair,URC*3
2,80692947,MOMO,2022-09-01,Off-take,Supply,0.0,2022-09-01,FAB ULTRAR CRM 56.7G(2.0OZ)X6X13 TW,Cream,2.0,Oz,78,Ultra Repair,URC*3
3,80692947,MOMO,2022-10-01,Off-take,Supply,1634.0,2022-10-01,FAB ULTRAR CRM 56.7G(2.0OZ)X6X13 TW,Cream,2.0,Oz,78,Ultra Repair,URC*3
4,80692947,MOMO,2022-11-01,Off-take,Supply,1285.0,2022-11-01,FAB ULTRAR CRM 56.7G(2.0OZ)X6X13 TW,Cream,2.0,Oz,78,Ultra Repair,URC*3


In [58]:
procure_plan_china = pd.read_excel('Replenishment_China.xlsx', sheet_name='Procurement ')
procure_plan_china =  procure_plan_china[[ 'scenario_name', 'operation_name', 'item_name', 'from_site_name', 'to_site_name',   'start_period_date','end_period_date', 'value']]
procure_plan_china.rename(columns = {'item_name' : 'sku'}, inplace = True)
procure_plan_china['date'] = procure_plan_china['start_period_date']
procure_plan_china = pd.merge(procure_plan_china, att_main_china, on = 'sku')

procure_plan_china['from_site_main'] = 'BSLY'
procure_plan_china.loc[procure_plan_china['from_site_name'].str.contains('PG'),'from_site_main'] = 'P&G'
procure_plan_china['to_site_main'] = 'BSLY'
procure_plan_china.loc[procure_plan_china['to_site_name'].str.contains('PG'),'to_site_main'] = 'PG'
procure_plan_china.loc[procure_plan_china['to_site_name'].str.contains('DY'),'to_site_main'] = 'DY'
procure_plan_china.loc[procure_plan_china['to_site_name'].str.contains('JD'),'to_site_main'] = 'JD'
procure_plan_china.loc[procure_plan_china['to_site_name'].str.contains('TMALL'),'to_site_main'] = 'TMALL'

procure_plan_china.head()

,scenario_name,operation_name,sku,from_site_name,to_site_name,start_period_date,end_period_date,value,date,sku_description,item_type,size,uom,cases,ph_level2,shortname,material,from_site_main,to_site_main
0,base,Procure:815517026036:Xiqing_PG:Shenyang_PG,815517026036,Xiqing_PG,Shenyang_PG,2022-07-01,2022-07-01,26,2022-07-01,FAB UT RPAR CRM 2.0OZ 56.7GX24 JAR JUL20,Cream,56.7,G,24,Ultra Repair,Facial Radiance Pads 28 pcs,82314745,P&G,PG
1,base,Procure:815517026036:Luogang_PG:Luogang_DY,815517026036,Luogang_PG,Luogang_DY,2022-07-01,2022-07-01,2616,2022-07-01,FAB UT RPAR CRM 2.0OZ 56.7GX24 JAR JUL20,Cream,56.7,G,24,Ultra Repair,Facial Radiance Pads 28 pcs,82314745,P&G,DY
2,base,Procure:815517026036:Xiqing_PG:Xiqing_DY,815517026036,Xiqing_PG,Xiqing_DY,2022-07-01,2022-07-01,726,2022-07-01,FAB UT RPAR CRM 2.0OZ 56.7GX24 JAR JUL20,Cream,56.7,G,24,Ultra Repair,Facial Radiance Pads 28 pcs,82314745,P&G,DY
3,base,Procure:815517026036:Shenyang_PG:Shenyang_JD,815517026036,Shenyang_PG,Shenyang_JD,2022-07-01,2022-07-01,48,2022-07-01,FAB UT RPAR CRM 2.0OZ 56.7GX24 JAR JUL20,Cream,56.7,G,24,Ultra Repair,Facial Radiance Pads 28 pcs,82314745,P&G,JD
4,base,Procure:815517026036:Wuhan_PG:Wuhan_JD,815517026036,Wuhan_PG,Wuhan_JD,2022-07-01,2022-07-01,88,2022-07-01,FAB UT RPAR CRM 2.0OZ 56.7GX24 JAR JUL20,Cream,56.7,G,24,Ultra Repair,Facial Radiance Pads 28 pcs,82314745,P&G,JD


In [65]:
procure_plan_taiwan = pd.read_excel('Replenishment Taiwan.xlsx', sheet_name='Procurement plan')
procure_plan_taiwan =  procure_plan_taiwan[[ 'scenario_name', 'operation_name', 'item_name', 'from_site_name', 'to_site_name',   'start_period_date','end_period_date', 'value']]
procure_plan_taiwan.rename(columns = {'item_name' : 'sku'}, inplace = True)
procure_plan_taiwan['date'] = procure_plan_taiwan['start_period_date']
procure_plan_taiwan = pd.merge(procure_plan_taiwan, att_main_tw, on = 'sku')
procure_plan_taiwan.head()

,scenario_name,operation_name,sku,from_site_name,to_site_name,start_period_date,end_period_date,value,date,sku_description,item_type,size,uom,cases,ph_level2,shortname
0,base,Procure:80702886:US_Supplier:PG_Warehouse,80702886,US_Supplier,PG_Warehouse,2023-01-01,2023-04-01,87,2023-01-01,FAB CLEANSER EUP 142G(50Z)X57 NONSTK,Cleanser,5.0,Oz,57,NaN,Cleanser
1,base,Procure:80702886:US_Supplier:PG_Warehouse,80702886,US_Supplier,PG_Warehouse,2023-02-01,2023-05-01,207,2023-02-01,FAB CLEANSER EUP 142G(50Z)X57 NONSTK,Cleanser,5.0,Oz,57,NaN,Cleanser
2,base,Procure:80702886:US_Supplier:PG_Warehouse,80702886,US_Supplier,PG_Warehouse,2023-03-01,2023-06-01,142,2023-03-01,FAB CLEANSER EUP 142G(50Z)X57 NONSTK,Cleanser,5.0,Oz,57,NaN,Cleanser
3,base,Procure:80702886:PG_Warehouse:MOMO,80702886,PG_Warehouse,MOMO,2022-11-01,2022-11-01,41,2022-11-01,FAB CLEANSER EUP 142G(50Z)X57 NONSTK,Cleanser,5.0,Oz,57,NaN,Cleanser
4,base,Procure:80702886:PG_Warehouse:MOMO,80702886,PG_Warehouse,MOMO,2022-12-01,2022-12-01,27,2022-12-01,FAB CLEANSER EUP 142G(50Z)X57 NONSTK,Cleanser,5.0,Oz,57,NaN,Cleanser


In [66]:
write_data_table.send_data(procure_plan_taiwan,'procure_plan_taiwan','w6')
write_data_table.send_data(procure_plan_china,'procure_plan_china','w7')
write_data_table.send_data(buffer_plan_taiwan,'buffer_plan_taiwan','w6')
write_data_table.send_data(buffer_plan_china,'buffer_plan_china','w7')

Index(['scenario_name', 'operation_name', 'sku', 'from_site_name', 'to_site_name', 'start_period_date', 'end_period_date', 'value', 'date', 'sku_description', 'item_type', 'size', 'uom', 'cases', 'ph_level2', 'shortname'], dtype='object')
Index(['scenario_name', 'operation_name', 'sku', 'from_site_name', 'to_site_name', 'start_period_date', 'end_period_date', 'value', 'date', 'sku_description', 'item_type', 'size', 'uom', 'cases', 'ph_level2', 'shortname', 'material', 'from_site_main', 'to_site_main'], dtype='object')
Index(['sku', 'site_name', 'period_start_date', 'Type', 'metric_name', 'metric_value', 'date', 'sku_description', 'item_type', 'size', 'uom', 'cases', 'ph_level2', 'shortname'], dtype='object')
Index(['sku', 'site_name', 'period_start_date', 'Type', 'metric_name', 'metric_value', 'date', 'sku_description', 'item_type', 'size', 'uom', 'cases', 'ph_level2', 'shortname', 'material', 'site_name_main'], dtype='object')


In [61]:
procure_plan_taiwan.dtypes

scenario_name                object
operation_name               object
sku                           int64
from_site_name               object
to_site_name                 object
start_period_date             int64
end_period_date      datetime64[ns]
value                         int64
date                          int64
sku_description              object
item_type                    object
size                        float64
uom                          object
cases                         int64
ph_level2                    object
shortname                    object
dtype: object